In [1]:
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Embedding, SimpleRNN, LSTM, Bidirectional
from tensorflow.keras.callbacks import History

In [3]:
data = pd.read_csv('events_filtered.csv')
data.head()

,Unnamed: 0,id_odsp,sort_order,time,event_type,event_type2,side,event_team,opponent,player,...,assist_method,fast_break,open_play,set_piece,corner,free_kick,is_team1_goal,is_team2_goal,team1_cumgoals,team2_cumgoals
0,580478,004f4ING/,1,1,8,0.0,2,83,88,24.0,...,0,0,0,0,0,0,False,False,0,0
1,580479,004f4ING/,2,1,3,0.0,1,88,83,2232.0,...,0,0,0,0,0,0,False,False,0,0
2,580480,004f4ING/,3,5,8,0.0,1,88,83,590.0,...,0,0,0,0,0,0,False,False,0,0
3,580481,004f4ING/,4,5,3,0.0,2,83,88,1199.0,...,0,0,0,0,0,0,False,False,0,0
4,580482,004f4ING/,5,6,1,12.0,1,88,83,988.0,...,2,0,1,0,0,0,False,False,0,0


In [ ]:
data.columns

Index(['Unnamed: 0', 'id_odsp', 'sort_order', 'time', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'is_goal', 'location',
       'bodypart', 'assist_method', 'fast_break', 'open_play', 'set_piece',
       'corner', 'free_kick', 'is_team1_goal', 'is_team2_goal',
       'team1_cumgoals', 'team2_cumgoals'],
      dtype='object')

In [ ]:
X_train = data.loc[data.id_odsp < 'p'][['sort_order', 'time', 'event_type', 'event_type2', 'side',
       'event_team', 'opponent', 'player', 'player2', 'player_in',
       'player_out', 'shot_place', 'location', 'bodypart',
       'assist_method', 'fast_break', 'open_play', 'set_piece', 'corner',
       'free_kick']]
y_train = data.loc[data.id_odsp < 'p'][['team1_cumgoals', 'team2_cumgoals']]
X_valid = data.loc[(data.id_odsp >= 'p').__and__(data.id_odsp < 'u')][['sort_order', 'time', 'event_type', 'event_type2', 'side',
       'event_team', 'opponent', 'player', 'player2', 'player_in',
       'player_out', 'shot_place', 'location', 'bodypart',
       'assist_method', 'fast_break', 'open_play', 'set_piece', 'corner',
       'free_kick']]
y_valid = data.loc[(data.id_odsp >= 'p').__and__(data.id_odsp < 'u')][['team1_cumgoals', 'team2_cumgoals']]
X_test = data.loc[data.id_odsp >= 'u'][['sort_order', 'time', 'event_type', 'event_type2', 'side',
       'event_team', 'opponent', 'player', 'player2', 'player_in',
       'player_out', 'shot_place', 'location', 'bodypart',
       'assist_method', 'fast_break', 'open_play', 'set_piece', 'corner',
       'free_kick']]
y_test = data.loc[data.id_odsp >= 'u'][['team1_cumgoals', 'team2_cumgoals']]

In [ ]:
history = History()
model = Sequential()
model.add(Dense(180, activation='ELU'))
model.add(Dropout(0.4))
model.add(Dense(20, activation='ELU'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='ELU'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)
model.fit(X_train, y_train, validation_data=[X_valid, y_valid], epochs=100, batch_size=15, callbacks=[history, early_stopping])

Epoch 1/100
50899/50899 [==============================] - 125s 2ms/step - loss: 7.1421 - accuracy: 0.7211 - val_loss: 0.8374 - val_accuracy: 0.7599
Epoch 2/100
50899/50899 [==============================] - 133s 3ms/step - loss: 0.8443 - accuracy: 0.7502 - val_loss: 0.8422 - val_accuracy: 0.7599
Epoch 3/100
50899/50899 [==============================] - 106s 2ms/step - loss: 0.8444 - accuracy: 0.7504 - val_loss: 0.8368 - val_accuracy: 0.7599
Epoch 4/100
50899/50899 [==============================] - 105s 2ms/step - loss: 0.8442 - accuracy: 0.7491 - val_loss: 0.8378 - val_accuracy: 0.7599
Epoch 5/100
50899/50899 [==============================] - 91s 2ms/step - loss: 0.8443 - accuracy: 0.7498 - val_loss: 0.8388 - val_accuracy: 0.7599
Epoch 6/100
50899/50899 [==============================] - 90s 2ms/step - loss: 0.8443 - accuracy: 0.7488 - val_loss: 0.8406 - val_accuracy: 0.7599
Epoch 7/100
50899/50899 [==============================] - 99s 2ms/step - loss: 0.8444 - accuracy: 0.7506 - 

In [ ]:
model.evaluate(X_test, y_test)

2721/2721 [==============================] - 4s 1ms/step - loss: 0.8723 - accuracy: 0.7520


[0.8723264932632446, 0.7520155906677246]

In [ ]:
model.predict(X_test.loc[X_test.time == 1])
model.predict(X_test.loc[X_test.time == 90])

array([[0.7689818 , 0.57918173],
       [0.7689818 , 0.57918173],
       [0.7689818 , 0.57918173],
       ...,
       [0.7689818 , 0.57918173],
       [0.7689818 , 0.57918173],
       [0.7689818 , 0.57918173]], dtype=float32)

In [ ]:
import os
import dill
if not os.path.exists('model.ipynb.1.db'):
    dill.dump_session('model.ipynb.1.db')
dump_checkpoint=1

INFO:tensorflow:Assets written to: ram://f9d7d979-28bb-49cc-8af8-97dc28c643d1/assets


INFO:tensorflow:Assets written to: ram://f9d7d979-28bb-49cc-8af8-97dc28c643d1/assets
